# **Clase ternaria**

# *Importación de librerías*

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sklearn as sk
from sklearn import model_selection
from sklearn import ensemble
from sklearn import metrics

# *Lectura de datos*

In [2]:
# from google.colab import drive
# drive.mount("/content/drive")

# DIR = "/content/drive/MyDrive/Maestría en Explotación de datos/Data Mining/TP de clasificación"
DIR = 'D:\Robin\DMEyF\datasets'

In [3]:
# Carga de df_train y creación de columna id
df_dataset = pd.read_csv(f'{DIR}/competencia_01_crudo.csv')
# df_dataset.set_index('id', inplace=True)
df_dataset.head()

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo
0,249221109,202101,1,0,0,61,300,1664.94,17994.40,1605.81,...,0.0,4.0,0.0,-37098.90,0.0,7867.0,118514.88,10.0,0.0,76538.25
1,249221468,202101,1,0,0,53,37,4879.78,18640.95,1149.08,...,0.0,4.0,0.0,-9864.93,0.0,1124.0,31427.14,14.0,0.0,0.00
2,249223005,202101,1,0,0,48,202,1828.35,37789.01,610.12,...,0.0,4.0,0.0,-32119.08,0.0,3486.0,855.11,2.0,0.0,19389.69
3,249228180,202101,1,0,0,66,320,175.87,2600.67,276.22,...,0.0,4.0,0.0,-10891.30,0.0,8925.0,7517.73,6.0,0.0,516.12
4,249232117,202101,1,0,0,79,373,16600.69,32270.07,16367.13,...,0.0,4.0,0.0,-43610.75,0.0,8607.0,37309.25,5.0,0.0,3084.99


In [9]:
def asignar_clase_ternaria(df_dataset, anio_mes_cero):
    
    anio_mes_cero_date = pd.to_datetime(int(anio_mes_cero), format='%Y%m')
    anio_mes_uno_date = anio_mes_cero_date + pd.DateOffset(months=1)
    anio_mes_dos_date = anio_mes_cero_date + pd.DateOffset(months=2)
    anio_mes_uno = int(anio_mes_uno_date.strftime('%Y%m'))
    anio_mes_dos = int(anio_mes_dos_date.strftime('%Y%m'))

    df_mes0 = df_dataset.loc[df_dataset['foto_mes'] == anio_mes_cero, ['numero_de_cliente', 'foto_mes']]
    df_mes0.set_index('numero_de_cliente', inplace=True)
    df_mes1 = df_dataset.loc[df_dataset['foto_mes'] == anio_mes_uno, ['numero_de_cliente', 'foto_mes']]
    df_mes1.set_index('numero_de_cliente', inplace=True)
    df_mes2 = df_dataset.loc[df_dataset['foto_mes'] == anio_mes_dos, ['numero_de_cliente', 'foto_mes']]
    df_mes2.set_index('numero_de_cliente', inplace=True)
        
    # Crear un DataFrame combinado con todos los índices
    all_clients = df_mes0.index.union(df_mes1.index).union(df_mes2.index)

    # Reindexar cada DataFrame para que tengan el mismo índice
    df_mes0_reindexed = df_mes0.reindex(all_clients)
    df_mes1_reindexed = df_mes1.reindex(all_clients)
    df_mes2_reindexed = df_mes2.reindex(all_clients)

    # Crear nuevas columnas para indicar la presencia en cada mes
    df_combined = pd.DataFrame(index=all_clients)
    df_combined['mes0'] = df_mes0_reindexed['foto_mes'].notna().astype(int)
    df_combined['mes1'] = df_mes1_reindexed['foto_mes'].notna().astype(int)
    df_combined['mes2'] = df_mes2_reindexed['foto_mes'].notna().astype(int)

    # Opcionalmente, si deseas mantener el formato de los DataFrames originales
    df_combined = df_combined.reset_index().rename(columns={'index': 'numero_de_cliente'})


    sum_mes1 = df_combined['mes1'].sum()
    sum_mes2 = df_combined['mes2'].sum()
    if sum_mes1 == 0 or sum_mes2 == 0:
        df_combined['clase_ternaria'] = np.nan
    
    else:
        # Definir las condiciones
        conditions = [
            (df_combined['mes0'] == 1) & (df_combined['mes1'] == 1) & (df_combined['mes2'] == 1),
            (df_combined['mes0'] == 1) & (df_combined['mes1'] == 1) & (df_combined['mes2'] == 0),
            (df_combined['mes0'] == 1) & (df_combined['mes1'] == 0) & (df_combined['mes2'] == 0)
        ]
        # Definir los valores que corresponden a cada condición
        values = ['CONTINUA', 'BAJA+2', 'BAJA+1']
        # Aplicar np.select para crear la nueva columna
        df_combined['clase_ternaria'] = np.select(conditions, values, default='OTRO CASO')

    # Crear un mapeo de numero_de_cliente a clase_ternaria
    cliente_a_clase = df_combined.set_index('numero_de_cliente')['clase_ternaria']

    # Asignar la clase_ternaria basada en el mapeo
    df_dataset.loc[df_dataset['foto_mes'] == anio_mes_cero, 'clase_ternaria'] = df_dataset.loc[df_dataset['foto_mes'] == anio_mes_cero, 'numero_de_cliente'].map(cliente_a_clase)

    return df_dataset

In [10]:
df_dataset = asignar_clase_ternaria(df_dataset, 202101)
df_dataset = asignar_clase_ternaria(df_dataset, 202102)
df_dataset = asignar_clase_ternaria(df_dataset, 202103)
df_dataset = asignar_clase_ternaria(df_dataset, 202104)
df_dataset = asignar_clase_ternaria(df_dataset, 202105)
df_dataset = asignar_clase_ternaria(df_dataset, 202106)

In [6]:
# df_dataset
df_dataset.groupby('foto_mes')['clase_ternaria'].value_counts().reset_index(name='count')


,foto_mes,clase_ternaria,count
0,202101,CONTINUA,160605
1,202101,BAJA+2,785
2,202101,BAJA+1,635
3,202101,OTRO CASO,1
4,202102,CONTINUA,160839
5,202102,BAJA+2,1017
6,202102,BAJA+1,785
7,202102,OTRO CASO,5
8,202103,CONTINUA,161681
9,202103,BAJA+1,1020


In [11]:
df_dataset.groupby('foto_mes')['clase_ternaria'].value_counts().reset_index(name='count')

,foto_mes,clase_ternaria,count
0,202101,CONTINUA,160605
1,202101,BAJA+2,785
2,202101,BAJA+1,635
3,202101,OTRO CASO,1
4,202102,CONTINUA,160839
5,202102,BAJA+2,1017
6,202102,BAJA+1,785
7,202102,OTRO CASO,5
8,202103,CONTINUA,161681
9,202103,BAJA+1,1020


In [13]:
df_dataset.loc[df_dataset['foto_mes'] == 202106]

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
817070,249221109,202106,1,0,0,62,305,399.88,20332.88,2115.35,...,0.0,0.0,-20783.64,0.0,8017.0,16701.69,9.0,0.0,1313.76,NaN
817071,249221468,202106,1,0,0,53,42,4708.37,23227.25,5537.14,...,0.0,0.0,-124338.00,0.0,1274.0,81981.26,28.0,0.0,0.00,NaN
817072,249223005,202106,1,0,0,49,207,3534.50,38207.62,534.72,...,0.0,0.0,-28621.20,0.0,3636.0,2669.90,3.0,0.0,18885.30,NaN
817073,249228180,202106,1,0,0,66,325,-138.82,1445.30,417.78,...,0.0,0.0,-17595.00,0.0,9075.0,20496.15,6.0,0.0,1818.15,NaN
817074,249232117,202106,1,0,0,80,378,556.99,36468.62,484.57,...,0.0,0.0,-48423.34,0.0,8757.0,44938.95,7.0,0.0,4058.58,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
981941,1598444941,202106,0,0,0,27,1,0.00,0.00,0.00,...,NaN,0.0,NaN,NaN,15.0,NaN,NaN,NaN,0.00,NaN
981942,1598471047,202106,1,0,0,28,1,95.20,95.20,0.00,...,7.0,0.0,NaN,NaN,14.0,NaN,NaN,NaN,0.00,NaN
981943,1598515897,202106,0,0,0,30,1,72.08,72.08,0.00,...,7.0,0.0,NaN,NaN,8.0,NaN,NaN,NaN,0.00,NaN
981944,1598517059,202106,0,0,0,24,1,0.00,0.00,0.00,...,0.0,0.0,NaN,NaN,1.0,NaN,NaN,NaN,0.00,NaN


In [14]:
# Exportar el DataFrame a un archivo CSV
# df_dataset.to_csv('dataset/competencia_01.csv', index=False)

### *Limpieza de datos*

In [3]:

# DIR = 'D:\Robin\DMEyF\datasets'
# df_dataset = pd.read_csv(f'{DIR}/competencia_01.csv')

C:\Users\Usuario\AppData\Local\Temp\ipykernel_3128\680918470.py:2: DtypeWarning: Columns (154) have mixed types. Specify dtype option on import or set low_memory=False.
  df_dataset = pd.read_csv(f'{DIR}/competencia_01.csv')


In [5]:
##Eliminar las columnas cprestamos_personales y mprestamos_personales de df_dataset

df_dataset.drop(columns=['cprestamos_personales', 'mprestamos_personales'], inplace=True)
df_dataset

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
0,249221109,202101,1,0,0,61,300,1664.94,17994.40,1605.81,...,4.0,0.0,-37098.90,0.0,7867.0,118514.88,10.0,0.0,76538.25,CONTINUA
1,249221468,202101,1,0,0,53,37,4879.78,18640.95,1149.08,...,4.0,0.0,-9864.93,0.0,1124.0,31427.14,14.0,0.0,0.00,CONTINUA
2,249223005,202101,1,0,0,48,202,1828.35,37789.01,610.12,...,4.0,0.0,-32119.08,0.0,3486.0,855.11,2.0,0.0,19389.69,CONTINUA
3,249228180,202101,1,0,0,66,320,175.87,2600.67,276.22,...,4.0,0.0,-10891.30,0.0,8925.0,7517.73,6.0,0.0,516.12,CONTINUA
4,249232117,202101,1,0,0,79,373,16600.69,32270.07,16367.13,...,4.0,0.0,-43610.75,0.0,8607.0,37309.25,5.0,0.0,3084.99,CONTINUA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
981941,1598444941,202106,0,0,0,27,1,0.00,0.00,0.00,...,NaN,0.0,NaN,NaN,15.0,NaN,NaN,NaN,0.00,NaN
981942,1598471047,202106,1,0,0,28,1,95.20,95.20,0.00,...,7.0,0.0,NaN,NaN,14.0,NaN,NaN,NaN,0.00,NaN
981943,1598515897,202106,0,0,0,30,1,72.08,72.08,0.00,...,7.0,0.0,NaN,NaN,8.0,NaN,NaN,NaN,0.00,NaN
981944,1598517059,202106,0,0,0,24,1,0.00,0.00,0.00,...,0.0,0.0,NaN,NaN,1.0,NaN,NaN,NaN,0.00,NaN


In [7]:
# Exportar el DataFrame a un archivo CSV
df_dataset.to_csv(f'{DIR}/competencia_01.csv')